In [ ]:
import sys
import os
from glob import glob

import numpy as np
import matplotlib.pyplot as plt

import pydicom

from pymedphys.gamma import gamma_shell, GammaInternalFixedOptions, calculate_min_dose_difference, convert_to_percent_pass

In [ ]:
DATA_DIRECTORY = "../../../tests/data/gamma/agnew_mcgarry_images"
glob(DATA_DIRECTORY)

In [ ]:
REF_VMAT_1mm = os.path.abspath(os.path.join(
    DATA_DIRECTORY, 'H&N_VMAT_Reference_1mmPx.dcm'))
EVAL_VMAT_1mm = os.path.abspath(os.path.join(
    DATA_DIRECTORY, 'H&N_VMAT_Evaluated_1mmPx.dcm'))

In [ ]:
def load_dose_from_dicom(dcm):
    pixels = dcm.pixel_array
    dose = pixels * dcm.DoseGridScaling

    return dose


def load_yx_from_dicom(dcm):
    resolution = np.array(
        dcm.PixelSpacing).astype(float)
    dx = resolution[0]
    x = (
        dcm.ImagePositionPatient[0] +
        np.arange(0, dcm.Columns * dx, dx))

    dy = resolution[1]
    y = (
        dcm.ImagePositionPatient[1] +
        np.arange(0, dcm.Rows * dy, dy))

    return y, x

In [ ]:
dcm_ref = pydicom.read_file(REF_VMAT_1mm)
dcm_eval = pydicom.read_file(EVAL_VMAT_1mm)

coords_reference = load_yx_from_dicom(dcm_ref)
dose_reference = load_dose_from_dicom(dcm_ref)

coords_evaluation = load_yx_from_dicom(dcm_eval)
dose_evaluation = load_dose_from_dicom(dcm_eval)

In [ ]:
%%timeit

dose_thresholds = 1
distance_thresholds = 1

gamma = gamma_shell(
    coords_reference, dose_reference,
    coords_evaluation, dose_evaluation,
    dose_thresholds, distance_thresholds, 
    max_gamma=1.0001, skip_once_passed=True, quiet=True)

In [ ]:
dose_thresholds = np.arange(1,3,0.1)
distance_thresholds = np.arange(1,3,0.1)

print("Num Gamma threshold combos = {}".format(len(dose_thresholds) * len(distance_thresholds)))

In [ ]:
%%timeit

gamma = gamma_shell(
    coords_reference, dose_reference,
    coords_evaluation, dose_evaluation,
    dose_thresholds, distance_thresholds, 
    max_gamma=1.0001, skip_once_passed=True, quiet=True)

In [ ]:
9.76 / 0.745

In [ ]:
pass_rates = []

for key, value in gamma.items():
    pass_rates.append(convert_to_percent_pass(value))
    
pass_rates = np.reshape(pass_rates, (len(dose_thresholds), len(distance_thresholds)))
    
plt.contourf(distance_thresholds, dose_thresholds, pass_rates, 100, vmin=90, vmax=100, cmap='coolwarm_r')

plt.title('Gamma pass-rate space')
plt.xlabel('Distance Threshold (mm)')
plt.ylabel('Dose Threshold (%)')
plt.colorbar(label='Gamma pass rate (gamma <= 1)')